# Data Migration and data mapping for financial services

Use the following code to map source and target schemas for data migration. 


In [2]:
import boto3
import os
from IPython.display import Markdown, display
from botocore.exceptions import ClientError
import logging


In [3]:
# Environmental Variables and Logging

region = os.environ.get("AWS_REGION")

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
    
logging.basicConfig(level=logging.INFO,format="%(levelname)s: %(message)s")


In [4]:
#bedrock_service = boto3.client(
#    service_name='bedrock',
#    region_name=region,
#)

In [5]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)

In [28]:
claude3 = 'claude3'
claude35 = 'claud35'
llama2 = 'llama2'
llama3='llama3'
mistral='mistral'
titan='titan'

models_dict = {
    claude3 : 'anthropic.claude-3-sonnet-20240229-v1:0',
    claude35: 'anthropic.claude-3-5-sonnet-20240620-v1:0',
    llama2: 'meta.llama2-13b-chat-v1',
    llama3: 'meta.llama3-8b-instruct-v1:0',
    mistral: 'mistral.mistral-7b-instruct-v0:2',
    titan : 'amazon.titan-text-premier-v1:0'
}

max_tokens_val = 100
temperature_val = 0.1

dict_add_params = {
    llama3: {}, #"max_gen_len":max_tokens_val, "temperature":temperature_val} , 
    claude3: {"top_k": 200, },# "temperature": temperature_val, "max_tokens": max_tokens_val},
    mistral: {}, #{"max_tokens":max_tokens_val, "temperature": temperature_val} , 
    titan:  {"topK": 200, },# "maxTokenCount": max_tokens_val}
}

inference_config={
    "temperature": temperature_val,
    "maxTokens": max_tokens_val,
    "topP": 0.9
}


In [7]:
def generate_conversation(bedrock_client,model_id,system_text,input_text):
    """
    Sends a message to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_text (JSON) : The system prompt.
        input text : The input message.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    logger.info("Generating message with model %s", model_id)

    # Message to send.
    message = {
        "role": "user",
        "content": [{"text": input_text}]
    }
    messages = [message]
    system_prompts = [{"text" : system_text}]

    if model_id in [models_dict.get(mistral), models_dict.get(titan)]:
        system_prompts = [] # not supported

    # Inference parameters to use.
    temperature = 0.3
    top_k = 200
    max_tokens=2000

    #Base inference parameters to use.
    # - inference_config
    inference_config={
        "temperature": temperature,
        "maxTokens": max_tokens,
        "topP": 0.9
    }


    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=get_additional_model_fields(model_id)
    )

    return response



In [8]:
def get_additional_model_fields(modelId):

    return dict_add_params.get(modelId)
    #{"top_k": top_k, "max_tokens": max_tokens}}
    


In [9]:

def get_converse_output(response_obj):
    ret_messages=[]
    output_message = response['output']['message']
    role_out = output_message['role']

    for content in output_message['content']:
        ret_messages.append(content['text'])
        
    return ret_messages, role_out

In [10]:
# Read JSON file
# Open the file in read mode
source = ""
with open('./schemas/source_schema.json', 'r') as file:
    # Read the entire contents of the file
    source = file.read()


In [11]:
# Read JSON file
# Open the file in read mode
target = ""
with open('./schemas/target_schema.json', 'r') as file:
    # Read the entire contents of the file
    target = file.read()

In [12]:
# print(source)

In [13]:
# print(target)

In [14]:
modelId = models_dict.get(claude3) #claude3) #llama3) # mistral # titan

system_text = "You are an expert data mapping engineer with vast insurance domain experience. Your job is to map source data model with the target data model. You create a JSON mapping file to map source table/entity and columns with target entity/table and column."

input_text = f"Can you map Source data model with the target data model. The source data model is \n <source> {source}  </source> and target data models is \n <target> {target}  </target>"

# print(input_text)



In [15]:
response = generate_conversation(bedrock_runtime, modelId, system_text, input_text)

output_message = response['output']['message']

print(f"Role: {output_message['role']}")

for content in output_message['content']:
    print(f"Text: {content['text']}")

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")

# print(f"Finished generating text with model {modelId}.")


INFO:__main__:Generating message with model anthropic.claude-3-sonnet-20240229-v1:0


Role: assistant
Text: Here's the JSON mapping file to map the source data model with the target data model:

```json
{
  "policyMaster": "insurance_products",
  "policyMaster.policyType": "insurance_products.product_name",
  "policyMaster.coverageDetails": "insurance_products.product_details",
  "insuredPersons": "customers",
  "insuredPersons.firstName": "customers.first_name",
  "insuredPersons.lastName": "customers.last_name",
  "insuredPersons.dateOfBirth": "customers.dob",
  "paymentInformation": "payment_methods",
  "paymentInformation.paymentMethod": "payment_methods.payment_type",
  "paymentInformation.paymentDetails": "payment_methods.payment_details",
  "policies": "policies",
  "policies.policyNumber": "policies.policy_id",
  "policies.policyTypeId": "policies.product_id",
  "policies.insuredPersonId": "policies.customer_id",
  "policies.paymentInformationId": "policies.payment_method_id",
  "policies.premiumAmount": "policies.premium_amount",
  "policies.effectiveDate": "po

In [16]:
display(Markdown(get_converse_output(response)[0][0]))

Here's the JSON mapping file to map the source data model with the target data model:

```json
{
  "policyMaster": "insurance_products",
  "policyMaster.policyType": "insurance_products.product_name",
  "policyMaster.coverageDetails": "insurance_products.product_details",
  "insuredPersons": "customers",
  "insuredPersons.firstName": "customers.first_name",
  "insuredPersons.lastName": "customers.last_name",
  "insuredPersons.dateOfBirth": "customers.dob",
  "paymentInformation": "payment_methods",
  "paymentInformation.paymentMethod": "payment_methods.payment_type",
  "paymentInformation.paymentDetails": "payment_methods.payment_details",
  "policies": "policies",
  "policies.policyNumber": "policies.policy_id",
  "policies.policyTypeId": "policies.product_id",
  "policies.insuredPersonId": "policies.customer_id",
  "policies.paymentInformationId": "policies.payment_method_id",
  "policies.premiumAmount": "policies.premium_amount",
  "policies.effectiveDate": "policies.start_date",
  "policies.expirationDate": "policies.end_date",
  "policies.branchId": "policies.branch_id",
  "insuranceBranches": "branches",
  "insuranceBranches.branchId": "branches.branch_id",
  "insuranceBranches.branchName": "branches.branch_name",
  "insuranceBranches.branchLocation.address": "branches.address.street",
  "insuranceBranches.branchLocation.city": "branches.address.city",
  "insuranceBranches.branchLocation.state": "branches.address.state",
  "insuranceBranches.branchLocation.zipCode": "branches.address.zip"
}
```

In this mapping file, the keys on the left side represent the source data model properties, and the values on the right side represent the corresponding target data model properties. The mapping is done based on the similarity of the property names and their data types.

For example, `"policyMaster": "insurance_products"` maps the `policyMaster` array in the source data model to the `insurance_products` array in the target data model. Similarly, `"policyMaster.policyType": "insurance_products.product_name"` maps the `policyType` property in the source data model to the `product_name` property in the target data model.

Note that some property names have been modified to match the target data model, such as `paymentMethod` to `payment_type`, `dateOfBirth` to `dob`, and `branchLocation` to `address`.

In [17]:
modelId = models_dict.get(llama3) #claude3) #llama3) # mistral # titan

response = generate_conversation(bedrock_runtime, modelId, system_text, input_text)

#output_message = response['output']['message']

display(Markdown(get_converse_output(response)[0][0]))

#print(f"Role: {output_message['role']}")

#for content in output_message['content']:
#    print(f"Text: {content['text']}")

#    token_usage = response['usage']
#    print(f"Input tokens:  {token_usage['inputTokens']}")
#    print(f"Output tokens:  {token_usage['outputTokens']}")
#    print(f"Total tokens:  {token_usage['totalTokens']}")
#    print(f"Stop reason: {response['stopReason']}")

INFO:__main__:Generating message with model meta.llama3-8b-instruct-v1:0




Based on the provided source and target data models, I will create a JSON mapping file to map the source table/entity and columns with the target entity/table and column.

Here is the JSON mapping file:
```
{
  "mappings": [
    {
      "source": "policyMaster",
      "target": "insurance_products",
      "mapping": [
        {
          "source": "policyType",
          "target": "product_name"
        },
        {
          "source": "coverageDetails",
          "target": "product_details"
        }
      ]
    },
    {
      "source": "insuredPersons",
      "target": "customers",
      "mapping": [
        {
          "source": "firstName",
          "target": "first_name"
        },
        {
          "source": "lastName",
          "target": "last_name"
        },
        {
          "source": "dateOfBirth",
          "target": "dob"
        }
      ]
    },
    {
      "source": "paymentInformation",
      "target": "payment_methods",
      "mapping": [
        {
          "source": "paymentMethod",
          "target": "payment_type"
        },
        {
          "source": "paymentDetails",
          "target": "payment_details"
        }
      ]
    },
    {
      "source": "policies",
      "target": "policies",
      "mapping": [
        {
          "source": "policyNumber",
          "target": "policy_id"
        },
        {
          "source": "policyTypeId",
          "target": "product_id"
        },
        {
          "source": "insuredPersonId",
          "target": "customer_id"
        },
        {
          "source": "paymentInformationId",
          "target": "payment_method_id"
        },
        {
          "source": "premiumAmount",
          "target": "premium_amount"
        },
        {
          "source": "effectiveDate",
          "target": "start_date"
        },
        {
          "source": "expirationDate",
          "target": "end_date"
        },
        {
          "source": "branchId",
          "target": "branch_id"
        }
      ]
    },
    {
      "source": "insuranceBranches",
      "target": "branches",
      "mapping": [
        {
          "source": "branchId",
          "target": "branch_id"
        },
        {
          "source": "branchName",
          "target": "branch_name"
        },
        {
          "source": "branchLocation.address",
          "target": "address.street"
        },
        {
          "source": "branchLocation.address.city",
          "target": "address.city"
        },
        {
          "source": "branchLocation.address.state",
          "target": "address.state"
        },
        {
          "source": "branchLocation.address.zipCode",
          "target": "address.zip"
        }
      ]
    }
  ]
}
```
This mapping file maps the source data model to the target data model. It maps the `policyMaster` array to the `insurance_products` array, the `insuredPersons` array to the `customers` array, the `paymentInformation` array to the `payment_methods` array, the `policies` array to the `policies` array, and the `insuranceBranches` array to the `branches` array.

Note that some fields have been mapped to different fields in the target data model, and some fields have been mapped to nested objects.

### Another Example

In [18]:
# Read JSON file
# Open the file in read mode
source = ""
with open('./schemas/source_schema-1.json', 'r') as file:
    # Read the entire contents of the file
    source = file.read()

In [19]:
input_text = f"Can you map Source data model with the target data model. The source data model is \n <source> {source}  </source> and target data models is \n <target> {target}  </target>"


In [20]:
response = generate_conversation(bedrock_runtime, modelId, system_text, input_text)

output_message = response['output']['message']

print(f"Role: {output_message['role']}")

for content in output_message['content']:
    print(f"Text: {content['text']}")

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")


INFO:__main__:Generating message with model meta.llama3-8b-instruct-v1:0


Role: assistant
Text: 

Based on the provided source and target data models, I will create a JSON mapping file to map the source table/entity and columns with the target entity/table and column.

Here is the JSON mapping file:

```
{
  "mappings": [
    {
      "source": "policies",
      "target": "policies",
      "mapping": [
        {
          "source": "policyNumber",
          "target": "policy_id"
        },
        {
          "source": "policyType",
          "target": "product_id"
        },
        {
          "source": "effectiveDate",
          "target": "start_date"
        },
        {
          "source": "expirationDate",
          "target": "end_date"
        },
        {
          "source": "premium",
          "target": "premium_amount"
        },
        {
          "source": "branchId",
          "target": "branch_id"
        }
      ]
    },
    {
      "source": "persons",
      "target": "customers",
      "mapping": [
        {
          "source": "firstName",

In [29]:
modelId = models_dict.get(claude35) #claude3) #llama3) # mistral # titan

input_text = f"""Can you map Source data model with the target data model. The source data model is \n <source> {source}  </source> and target data models is \n <target> {target}  </target>. 
If you are unable to map then mention that you are not able to map instead of incorrectly mapping. 
"""


In [30]:
response = generate_conversation(bedrock_runtime, modelId, system_text, input_text)

output_message = response['output']['message']

print(f"Role: {output_message['role']}")

for content in output_message['content']:
    print(f"Text: {content['text']}")

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")


INFO:__main__:Generating message with model anthropic.claude-3-5-sonnet-20240620-v1:0


Role: assistant
Text: I can create a JSON mapping file to map the source data model with the target data model. Here's the mapping:

```json
{
  "mappings": [
    {
      "source": {
        "entity": "policies",
        "attributes": {
          "policyNumber": "policy_id",
          "policyType": "product_id",
          "effectiveDate": "start_date",
          "expirationDate": "end_date",
          "premium": "premium_amount"
        }
      },
      "target": {
        "entity": "policies"
      }
    },
    {
      "source": {
        "entity": "persons",
        "attributes": {
          "firstName": "first_name",
          "lastName": "last_name",
          "dateOfBirth": "dob"
        }
      },
      "target": {
        "entity": "customers"
      }
    },
    {
      "source": {
        "entity": "branches",
        "attributes": {
          "branchId": "branch_id",
          "branchName": "branch_name",
          "address": "address"
        }
      },
      "target": {
    